In [18]:
from functools import reduce
import re
import pandas as pd
import numpy as np
from scipy.spatial.distance import mahalanobis
from scipy.optimize import linear_sum_assignment
import warnings

warnings.filterwarnings('ignore')

In [19]:
events1 = pd.read_csv('новые_протесты_рус3.csv')
events2 = pd.read_csv('росстат_общий_new.csv')


events1['Date'] = pd.to_datetime(events1['Date'], errors='coerce')
events1['year'] = events1['Date'].dt.year  
events1['Event text'] = 1

corrections = {
    "Кемерово":        ("Кемеровская",     "Кемеровский",   32701000),
    "Пермь":           ("Пермский",        "Пермский",      57701000),
    "Петрозаводск":    ("Карелия",         "Петрозаводский",86701000),
    "Казань":          ("Татарстан",       "Казань",        92701000),
    "Владивосток":     ("Приморский",      "Владивостокский",5701000),
    "Ростов":          ("Ярославская",     "Ростовский",    78637000),
    "Нижний Новгород": ("Нижегородская",   "Нижний Новгород",22701000),
    "Орёл":            ("Орловская",       "Орёл",          54701000),
    "Якутск":          ("Саха",            "Якутск",        98701000),
    "Барнаул":         ("Алтайский",       "Барнаул",       1701000),
    "Челябинск":       ("Челябинская",     "Челябинский",   75701000),
    "Магас":           ("Ингушетия",       "Магас",         26701000),
    "Черкесск":        ("Карачаево-Черкесская","Черкесский",91701000),
    "Элиста":          ("Калмыкия",        "Элиста",        85701000),
    "Тула":            ("Тульская",        "Тула",          70701000),
    "Кострома":        ("Костромская",     "Кострома",      34701000),
    "Грозный":         ("Чеченская",       "Грозный",       96701000),
    "Саранск":         ("Мордовия",        "Саранск",       89701000),
    "Волчанск":        ("Свердловская",    "Волчанский",    65735000),
    "Сочи":            ("Краснодарский",   "Сочи",          3726000),
    "Сыктывкар":       ("Коми",            "Сыктывкар",     87701000),
    "Магнитогорск":    ("Челябинская",     "Магнитогорский",75738000),
    "Чита":            ("Забайкальский",   "Чита",          76701000),
    "Невьянск":        ("Свердловская",    "Невьянский",    65714000),
    "Кормиловка":      ("Омская",          "Кормиловский",  52623000),
    "Златоуст":        ("Челябинская",     "Златоустовский",75712000),
    "Вологда":         ("Вологодская",     "Вологда",       19701000),
    "Киров-Чепецк":    ("Кировская",       "Кирово-Чепецк", 33707000),
    "Ростов-на-Дону":  ("Ростовская",      "Ростов-на-Дону",60701000),
    "Реж":             ("Свердловская",    "Режевской",     65720000),
    "Качканар":        ("Свердловская",    "Качканарский",  65743000),
    "Нижний Тагил":    ("Свердловская",    "Нижний Тагил",  65751000),
    "Бакал":           ("Башкортостан",    "Бакалинский",   80607000),
}

# 3) Применяем коррекцию
for wrong_name, (new_region, new_municipality, new_oktmo) in corrections.items():
    mask = events1['municipality'] == wrong_name
    events1.loc[mask, 'region_name']  = new_region
    events1.loc[mask, 'municipality'] = new_municipality
    events1.loc[mask, 'oktmo']        = new_oktmo





df = events2.merge(
    events1[['region_name','municipality', 'year', '# protesters', 'Event text']],
    on=['municipality', 'region_name', 'year'],
    how='left'
)


df = df[df['region_name'] != 'Москва'].reset_index(drop=True)
df = df[df['region_name'] != 'Санкт-Петербург'].reset_index(drop=True)
# df = df[df['region_name'] != 'Московская'].reset_index(drop=True)
df.to_csv('общие_данные_2010t3.csv', index=False)
df

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180968,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN,NaN
180969,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN,NaN
180970,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
180971,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN


In [20]:
events2 = pd.read_csv('общие_данные_2010t3.csv')
protests = events2[events2['Event text'] == 1]
protests_0910 = protests[protests['year'].isin([2011, 2012, 2013])].copy()

# Для каждого oktmo записываем минимальный год протеста в 2009–2010
protests_0910['fprotets_year'] = protests_0910.groupby('municipality')['year'].transform('min')

# Оставляем по одной строке на oktmo (колонка fprotets_year уже заполнена правильно)
protests1 = protests_0910.drop_duplicates(subset=['municipality']).reset_index(drop=True)



protests.to_csv('протесты_2010t3.csv', index=False)
protests1.to_csv('протесты2009_2013_2010t3.csv', index=False)
protests1

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text,fprotets_year
0,Абакан,95701000,79.594802,Хакасия,2012.0,2.1,0.00,180.0,1946.0,40.891433,100.000000,46335793.0,100.0,1.0,2012.0
1,Архангельск,11701000,0.000000,Архангельская,2011.0,538.2,2.94,2400.0,5900.0,0.000000,0.000000,85398442.0,200.0,1.0,2011.0
2,Астрахань,12701000,86.242222,Астраханская,2011.0,1655.9,38.00,2000.0,4000.0,39.121620,100.000000,97906547.0,0.0,1.0,2011.0
3,Барнаул,1701000,98.226081,Алтайский,2011.0,86.0,0.00,100.0,5400.0,41.532612,94.159013,178778211.0,0.0,1.0,2011.0
4,Белгород,14701000,96.058305,Белгородская,2011.0,139.1,0.00,0.0,100.0,0.000000,0.000000,82445277.0,0.0,1.0,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Чебоксары,97701000,34.551970,Чувашская,2011.0,96.3,0.00,2000.0,7800.0,0.000000,0.000000,81177377.0,100,1.0,2011.0
59,Челябинский,75701000,91.399734,Челябинская,2011.0,131.3,0.00,3400.0,14000.0,0.000000,0.000000,322677382.8,0,1.0,2011.0
60,Чита,76701000,93.486767,Забайкальский,2011.0,0.0,0.00,500.0,6400.0,24.369539,100.000000,0.0,0,1.0,2011.0
61,Южно-Сахалинск,64701000,77.693897,Сахалинская,2011.0,123.6,96.48,200.0,700.0,25.110739,96.155110,89051666.0,9,1.0,2011.0


In [21]:
lol = pd.read_csv("протесты_2010t3.csv") 
sobytia = pd.read_csv("общие_данные_2010t3.csv")

noprotestALL = sobytia[~sobytia['oktmo'].isin(lol['oktmo'])].reset_index(drop=True)
noprotestALL.to_csv('единицы_без_протестов_2010t3.csv', index=False)
noprotestALL

,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text
0,NaN,28602100,0.000000,Тверская,2010.0,1.9,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
1,NaN,28602100,0.000000,Тверская,2012.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
2,NaN,28602100,0.000000,Тверская,2014.0,2.2,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
3,NaN,28602100,0.000000,Тверская,2015.0,0.0,0.0,100.0,300.0,0.000000,0.0,0.0,NaN,NaN
4,NaN,28602400,0.000000,Тверская,2010.0,14.3,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176476,имени Полины Осипенко,8637000,91.832956,Хабаровский,2022.0,0.0,79.6,0.0,0.0,54.528488,0.0,0.0,NaN,NaN
176477,имени Полины Осипенко,8637000,91.832956,Хабаровский,2023.0,0.0,80.3,0.0,0.0,54.432127,0.0,0.0,NaN,NaN
176478,имени Полины Осипенко,8637406,93.960350,Хабаровский,2007.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN
176479,имени Полины Осипенко,8637406,93.960350,Хабаровский,2008.0,0.4,0.0,0.0,0.0,0.000000,0.0,0.0,NaN,NaN


In [22]:
# import pandas as pd

# # 1. Чтение данных
# df = pd.read_csv('протесты_2010t3.csv')


# # 3. Преобразуем количество протестующих в числовой формат
# df['# protesters'] = pd.to_numeric(df['# protesters'], errors='coerce').fillna(0)

# # 4. Фильтруем данные по годам 2009-2010 и 2007
# df_2009_2010 = df[df['year'].isin([2011, 2012, 2013])]
# df_2007 = df[df['year'] == 2010]

# # 5. Вычисляем среднее количество протестующих для периода 2009-2010
# mean_2009_2010 = df_2009_2010.groupby('municipality', as_index=False)['# protesters'].mean()
# mean_2009_2010.rename(columns={'# protesters': 'mean_protesters_2009_2010'}, inplace=True)

# # 6. Вычисляем среднее количество протестующих за 2007 год
# mean_2007 = df_2007.groupby('municipality', as_index=False)['# protesters'].mean()
# mean_2007.rename(columns={'# protesters': 'mean_protesters_2007'}, inplace=True)

# # 7. Объединяем результаты по municipal (oktmo)
# merged = pd.merge(mean_2009_2010, mean_2007, on='municipality', how='inner')

# # 8. Фильтруем муниципалитеты, где среднее количество протестующих за 2009-2010 больше, чем за 2007
# result = merged[merged['mean_protesters_2009_2010'] > merged['mean_protesters_2007']]

# # 9. Получаем исходные данные для выбранных муниципалитетов и удаляем дубликаты
# final_df = pd.merge(result['municipality'], df_2009_2010, on='municipality', how='inner').drop_duplicates('municipality').reset_index(drop=True)

# # 10. Сохраняем результаты в новый файл
# final_df.to_csv('без_протестов2009_2010t3.csv', index=False)
# final_df



In [ ]:
PRE_YEARS  = [2010]                   # препротестный период
POST_YEARS = [2011, 2012, 2013]       # протестный период
MIN_POST_EVENTS = 1
TARGET = 100


df = pd.read_csv("протесты_2010t3.csv" )
df['# protesters'] = pd.to_numeric(df['# protesters'], errors='coerce').fillna(0)
df['year'] = pd.to_numeric(df['year'], errors='coerce').astype('Int64')
df['event_flag'] = ((df.get('Event text', np.nan) == 1) | (df['# protesters'] > 0)).astype(int)

#  Агрегаты 
def aggregate_for_years(sub_years, prefix):
    sub = df[df['year'].isin(sub_years)].copy()
    agg = (
        sub.groupby('municipality', as_index=False)
           .agg(**{
               f'{prefix}_events': ('event_flag', 'sum'),
               f'{prefix}_sum':    ('# protesters', 'sum'),
               f'{prefix}_mean':   ('# protesters', 'mean')
           })
    )
    return agg

pre  = aggregate_for_years(PRE_YEARS,  'pre')
post = aggregate_for_years(POST_YEARS, 'post')
agg = pd.merge(pre, post, on='municipality', how='outer').fillna(0)

# Скор
agg['score'] = (
    0.50 * np.log((agg['post_mean'] + 1) / (agg['pre_mean'] + 1)) +
    0.30 * np.log1p(agg['post_sum']) -
    0.20 * np.log1p(agg['pre_sum'])
)

agg = agg[agg['post_events'] >= MIN_POST_EVENTS]



top_munis = (
    agg.sort_values('score', ascending=False)
       .head(TARGET)
       .loc[lambda x: x['score'] >= 0, 'municipality']
       .unique()
)


df_post = df[df['year'].isin(POST_YEARS)]
final_df = (
    df_post[df_post['municipality'].isin(top_munis)]
    .drop_duplicates(subset='municipality')
    .reset_index(drop=True)
)

print(f"Выбрано {final_df['municipality'].nunique()} муниципалитетов.")
final_df.to_csv('без_протестов2009_2010t3.csv', index=False)
final_df


Выбрано 42 муниципалитетов.


,municipality,oktmo,Процент (рус),region_name,year,Жилье (1000 м2),Дороги (%),Канализация (1 м),Водопровод (1 метр),active_percent,city_percent,value,# protesters,Event text,event_flag
0,Астрахань,12701000,86.242222,Астраханская,2011,1655.9,38.00,2000.0,4000.00,39.121620,100.000000,9.790655e+07,0.0,1.0,1
1,Белгород,14701000,96.058305,Белгородская,2011,139.1,0.00,0.0,100.00,0.000000,0.000000,8.244528e+07,0.0,1.0,1
2,Брянск,15701000,97.707806,Брянская,2011,98.2,0.00,1200.0,2500.00,40.492615,0.000000,7.933087e+07,0.0,1.0,1
3,Владивостокский,5701000,96.447139,Приморский,2011,99.4,0.00,0.0,2568.50,42.424584,95.968640,1.555050e+08,100.0,1.0,1
4,Владимир,17701000,48.991974,Владимирская,2012,55.6,34.16,0.0,500.00,46.436342,99.257368,8.130472e+07,0.0,1.0,1
5,Волгоград,18701000,97.113306,Волгоградская,2011,352.1,30.01,545.1,19324.30,0.000000,0.000000,2.197124e+08,20.0,1.0,1
6,Вологда,19701000,0.000000,Вологодская,2011,266.0,0.00,67.0,1070.00,40.882997,48.770540,8.159991e+07,0.0,1.0,1
7,Воронеж,20701000,98.485943,Воронежская,2011,182.6,0.00,500.0,3500.00,40.936929,100.000000,2.188282e+08,0.0,1.0,1
8,Грозный,96701000,3.382850,Чеченская,2011,0.0,0.00,0.0,0.00,0.000000,0.000000,2.889121e+07,40.0,1.0,1
9,Екатеринбург,65701000,94.934784,Свердловская,2011,258.0,0.00,500.0,4800.00,0.000000,0.000000,5.690012e+08,0.0,1.0,1


In [24]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)



df_protests = pd.read_csv("протесты2009_2013_2010t3.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t3.csv")
df_events_full = pd.read_csv("общие_данные_2010t3.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}

df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)

print("Размер df_protests:", len(df_protests))
print("Размер df_events:", len(df_events))

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()

if degenerate:
    print(f"Вырожденные признаки (дисперсия <= {threshold}): {degenerate}")
    features = [f for f in features if f not in degenerate]
    print(f"Оставшиеся признаки: {features}")
else:
    print("Нет вырожденных признаков.")

combined = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined.T)
cov_inv = np.linalg.inv(cov_matrix)

complete_event_oktmos = []
for oktmo in df_events['oktmo'].unique():
    if check_years_completeness(oktmo, df_events_full, required_years):
        complete_event_oktmos.append(oktmo)

df_events_complete = df_events[df_events['oktmo'].isin(complete_event_oktmos)].reset_index(drop=True)

print(f"После фильтрации по годам осталось {len(df_events_complete)} муниципалитетов событий")

n_protests = df_protests.shape[0]
n_events = df_events_complete.shape[0]
distance_matrix = np.zeros((n_protests, n_events))

for i in range(n_protests):
    x = df_protests.loc[i, features].values
    for j in range(n_events):
        y = df_events_complete.loc[j, features].values
        distance_matrix[i, j] = mahalanobis(x, y, cov_inv)

row_ind, col_ind = linear_sum_assignment(distance_matrix)

df_matched_protests = df_protests.iloc[row_ind].reset_index(drop=True)
df_matched_events = df_events_complete.iloc[col_ind].reset_index(drop=True)

df_matches = df_matched_protests.join(df_matched_events, lsuffix="_protest", rsuffix="_event")
df_matches["mahalanobis_distance"] = distance_matrix[row_ind, col_ind]

df_matches.to_csv('прототип_данных_2010t3.csv', index=False)
df_matches

Размер df_protests: 63
Размер df_events: 27290
Нет вырожденных признаков.
После фильтрации по годам осталось 8721 муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2012.0,2.1,0.00,180.0,1946.0,Краснодарский,Темрюкское,3651101,0.0,0.00,185.0,1988.0,0.012734
1,Архангельская,Архангельск,11701000,2011.0,538.2,2.94,2400.0,5900.0,Ленинградская,Выборгский,41615000,156.7,0.00,2438.0,6650.0,0.378307
2,Астраханская,Астрахань,12701000,2011.0,1655.9,38.00,2000.0,4000.0,Марий,Медведевский,88628000,279.5,33.88,1800.0,2427.0,1.059496
3,Алтайский,Барнаул,1701000,2011.0,86.0,0.00,100.0,5400.0,Тульская,Новомосковск,70724000,80.1,0.00,100.0,5460.0,0.017316
4,Белгородская,Белгород,14701000,2011.0,139.1,0.00,0.0,100.0,Тверская,Спировский,28652000,128.7,0.00,0.0,80.0,0.008992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,Чувашская,Чебоксары,97701000,2011.0,96.3,0.00,2000.0,7800.0,Кемеровская,Тяжинское,32634151,95.7,0.00,2000.0,7210.0,0.164785
59,Челябинская,Челябинский,75701000,2011.0,131.3,0.00,3400.0,14000.0,Костромская,Мантурово,34714000,0.0,0.00,4000.0,13300.0,0.917161
60,Забайкальский,Чита,76701000,2011.0,0.0,0.00,500.0,6400.0,Ростовская,Таганрог,60737000,52.5,0.00,400.0,6200.0,0.148911
61,Сахалинская,Южно-Сахалинск,64701000,2011.0,123.6,96.48,200.0,700.0,Ростовская,Батайск,60707000,5.0,95.44,100.0,600.0,0.167379


In [25]:
df_2012 = pd.read_csv('прототип_данных_2010t3.csv')
df_2011 = pd.read_csv('без_протестов2009_2010t3.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)


merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
    'region_name_protest', 
    'region_name_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t3.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_protest,region_name_event,fprotets_year
0,Астрахань,12701000,1655.9,38.00,2000.0,4000.00,Медведевский,88628000,279.50,33.88,1800.0,2427.0,Астраханская,Марий,2011.0
1,Белгород,14701000,139.1,0.00,0.0,100.00,Спировский,28652000,128.70,0.00,0.0,80.0,Белгородская,Тверская,2011.0
2,Брянск,15701000,98.2,0.00,1200.0,2500.00,Тоцкий,53652000,157.30,0.00,1200.0,2500.0,Брянская,Оренбургская,2011.0
3,Владивостокский,5701000,99.4,0.00,0.0,2568.50,Соль-Илецкий,53644000,90.44,0.00,0.0,2500.0,Приморский,Оренбургская,2011.0
4,Владимир,17701000,55.6,34.16,0.0,500.00,Хилокский,76647000,19.20,34.18,0.0,500.0,Владимирская,Забайкальский,2012.0
5,Волгоград,18701000,352.1,30.01,545.1,19324.30,Марьяновский,52630000,54.20,25.38,500.0,19000.0,Волгоградская,Омская,2011.0
6,Вологда,19701000,266.0,0.00,67.0,1070.00,Копейский,75728000,263.70,0.00,100.0,800.0,Вологодская,Челябинская,2011.0
7,Воронеж,20701000,182.6,0.00,500.0,3500.00,Рузаевка,89643101,0.00,0.00,500.0,3400.0,Воронежская,Мордовия,2011.0
8,Грозный,96701000,0.0,0.00,0.0,0.00,Агульский,82601000,0.00,0.00,0.0,0.0,Чеченская,Дагестан,2011.0
9,Екатеринбург,65701000,258.0,0.00,500.0,4800.00,Акбулакский,53605000,36.80,0.00,500.0,4880.0,Свердловская,Оренбургская,2011.0


In [26]:
df = pd.read_csv('общие_данные_2010t3.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',  'region_name',
        'Процент (рус)',
        ]]
df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])



# Фильтруем по годам 2011, 2013, 2014, 2015
df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
events = df
from2012 = pd.read_csv('связка_махаланобис_2010t3.csv')


final = pd.DataFrame()
for _, link_row in from2012.iterrows():
    

    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    

    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    

    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012, 2013]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t3.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,Процент (рус)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,Процент (рус)_event,fprotets_year
0,2008.0,Астрахань,12701000,1762.1,6.39,420.0,6906.0,Астраханская,86.242222,Медведевский,88628000.0,42.40,0.00,110.0,508.0,Марий,49.337173,NaN
1,2009.0,Астрахань,12701000,1739.0,1.08,8230.0,1200.0,Астраханская,86.242222,Медведевский,88628000.0,36.32,1.01,140.0,1434.0,Марий,49.337173,NaN
2,2010.0,Астрахань,12701000,1672.9,91.15,6600.0,8800.0,Астраханская,86.242222,Медведевский,88628000.0,46.70,0.00,57.0,1694.0,Марий,49.337173,NaN
3,2011.0,Астрахань,12701000,1655.9,38.00,2000.0,4000.0,Астраханская,86.242222,Медведевский,88628000.0,215.10,19.79,52.0,1690.0,Марий,49.337173,2011.0
4,2012.0,Астрахань,12701000,1647.4,38.00,6290.0,18033.0,Астраханская,86.242222,Медведевский,88628000.0,279.50,33.88,1800.0,2427.0,Марий,49.337173,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,2012.0,Ярославль,78701000,283.1,25.42,3200.0,12560.0,Ярославская,0.000000,Марковское,19620444.0,0.00,0.00,7200.0,13800.0,Вологодская,0.000000,2011.0
368,2013.0,Ярославль,78701000,205.7,38.68,2160.0,16690.0,Ярославская,0.000000,Марковское,19620444.0,0.70,0.00,7200.0,13800.0,Вологодская,0.000000,2011.0
369,2014.0,Ярославль,78701000,204.9,30.86,10150.0,22830.0,Ярославская,0.000000,Марковское,19620444.0,0.30,0.00,7200.0,13800.0,Вологодская,0.000000,NaN
370,2015.0,Ярославль,78701000,0.0,39.53,3500.0,11000.0,Ярославская,0.000000,Марковское,19620444.0,0.00,0.00,100.0,0.0,Вологодская,0.000000,NaN


ВТОРОЙ МАЧИНГ

In [27]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)

def matching_with_year_completeness(df_protests, df_events, df_events_full, features, required_years, cov_inv):
    matches_list = []
    
    for region in df_protests['region_name'].unique():
        sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
        sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
        
        complete_event_munis = []
        for _, row in sub_events.iterrows():
            if check_years_completeness(row['oktmo'], df_events_full, required_years):
                complete_event_munis.append(row['oktmo'])
        
        sub_events_complete = sub_events[sub_events['oktmo'].isin(complete_event_munis)].reset_index(drop=True)
        
        if len(sub_events_complete) == 0:
            print(f"В регионе {region} нет муниципалитетов событий с полными данными за все годы")
            continue
        
        n_p = sub_protests.shape[0]
        n_e = sub_events_complete.shape[0]
        dist_mat = np.zeros((n_p, n_e))
        
        for i in range(n_p):
            x = sub_protests.loc[i, features].values
            for j in range(n_e):
                y = sub_events_complete.loc[j, features].values
                dist_mat[i, j] = mahalanobis(x, y, cov_inv)
        
        row_ind, col_ind = linear_sum_assignment(dist_mat)
        
        matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
        matched_e = sub_events_complete.iloc[col_ind].reset_index(drop=True)
        matched = matched_p.join(
            matched_e,
            lsuffix="_protest",
            rsuffix="_event"
        )
        matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
        matches_list.append(matched)
    
    return matches_list

df_protests = pd.read_csv("протесты2009_2013_2010t3.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t3.csv")
df_events_full = pd.read_csv("общие_данные_2010t3.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}
df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)


features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[
    ['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features
].reset_index(drop=True)
df_events = df_events[
    ['region_name', 'municipality', 'oktmo'] + features
].reset_index(drop=True)

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

matches_list = matching_with_year_completeness(
    df_protests, df_events, df_events_full, features, required_years, cov_inv
)


df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t3M.csv', index=False)
df_matches
    


Вырожденных признаков нет.
В регионе Чеченская нет муниципалитетов событий с полными данными за все годы
В регионе Магаданская нет муниципалитетов событий с полными данными за все годы


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2012.0,2.1,0.00,180.0,1946.0,Хакасия,Первомайский,95615440,0.0,0.00,0.0,1866.0,0.254525
1,Архангельская,Архангельск,11701000,2011.0,538.2,2.94,2400.0,5900.0,Архангельская,Котласский,11627000,44.9,5.11,1352.0,3408.0,1.585701
2,Астраханская,Астрахань,12701000,2011.0,1655.9,38.00,2000.0,4000.0,Астраханская,Икрянинский,12620000,4.9,7.12,1300.0,6090.0,2.283757
3,Алтайский,Барнаул,1701000,2011.0,86.0,0.00,100.0,5400.0,Алтайский,Родинский,1636000,4.7,0.00,0.0,5200.0,0.154212
4,Белгородская,Белгород,14701000,2011.0,139.1,0.00,0.0,100.0,Белгородская,Алексеевский,14605000,61.3,0.00,0.0,100.0,0.053495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Ханты-Мансийский,Ханты-Мансийск,71871000,2011.0,83.0,0.00,0.0,1195.0,Ханты-Мансийский,Нижневартовский,71819000,39.4,0.00,0.0,1200.0,0.030038
57,Чувашская,Чебоксары,97701000,2011.0,96.3,0.00,2000.0,7800.0,Чувашская,Канаш,97707000,12.0,0.00,800.0,800.0,2.365169
58,Челябинская,Челябинский,75701000,2011.0,131.3,0.00,3400.0,14000.0,Челябинская,Розинское,75633156,0.0,0.00,4550.0,7900.0,2.567516
59,Забайкальский,Чита,76701000,2011.0,0.0,0.00,500.0,6400.0,Забайкальский,Забайкальский,76612000,0.8,7.81,800.0,2790.0,1.261249


In [28]:
df_2012 = pd.read_csv('прототип_данных_2010t3M.csv')
df_2011 = pd.read_csv('без_протестов2009_2010t3.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
    'region_name_protest', 
    'region_name_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t3М.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_protest,region_name_event,fprotets_year
0,Астрахань,12701000,1655.9,38.00,2000.0,4000.00,Икрянинский,12620000,4.90,7.12,1300.0,6090.0,Астраханская,Астраханская,2011.0
1,Белгород,14701000,139.1,0.00,0.0,100.00,Алексеевский,14605000,61.30,0.00,0.0,100.0,Белгородская,Белгородская,2011.0
2,Брянск,15701000,98.2,0.00,1200.0,2500.00,Новозыбковский,15640000,0.00,0.00,660.0,0.0,Брянская,Брянская,2011.0
3,Владивостокский,5701000,99.4,0.00,0.0,2568.50,Михайловский,5620000,1.19,0.00,0.0,2544.0,Приморский,Приморский,2011.0
4,Владимир,17701000,55.6,34.16,0.0,500.00,Округ,17735000,59.80,34.88,0.0,582.0,Владимирская,Владимирская,2012.0
5,Волгоград,18701000,352.1,30.01,545.1,19324.30,Старополтавский,18652000,0.10,0.00,0.0,21900.0,Волгоградская,Волгоградская,2011.0
6,Вологда,19701000,266.0,0.00,67.0,1070.00,Тотемский,19646000,49.00,0.00,50.0,1000.0,Вологодская,Вологодская,2011.0
7,Воронеж,20701000,182.6,0.00,500.0,3500.00,Панинский,20635000,4.40,0.00,400.0,5000.0,Воронежская,Воронежская,2011.0
8,Екатеринбург,65701000,258.0,0.00,500.0,4800.00,Горноуральский,65717000,10.80,0.00,600.0,4960.0,Свердловская,Свердловская,2011.0
9,Ижевск,94701000,44.3,64.61,600.0,2000.00,Глазовский,94610000,5.30,63.97,532.0,1430.0,Удмуртская,Удмуртская,2011.0


In [29]:
df = pd.read_csv('общие_данные_2010t3.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',  'region_name',
        'Процент (рус)',
        ]]
df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])


df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
events = df
from2012 = pd.read_csv('связка_махаланобис_2010t3М.csv')


final = pd.DataFrame()
for _, link_row in from2012.iterrows():
    

    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    

    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    

    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012, 2013]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t3М.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,Процент (рус)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,Процент (рус)_event,fprotets_year
0,2008.0,Астрахань,12701000,1762.1,6.39,420.0,6906.0,Астраханская,86.242222,Икрянинский,12620000.0,41.90,0.00,600.0,1632.0,Астраханская,85.066301,NaN
1,2009.0,Астрахань,12701000,1739.0,1.08,8230.0,1200.0,Астраханская,86.242222,Икрянинский,12620000.0,34.19,1.94,0.0,800.0,Астраханская,85.066301,NaN
2,2010.0,Астрахань,12701000,1672.9,91.15,6600.0,8800.0,Астраханская,86.242222,Икрянинский,12620000.0,28.60,0.00,407.0,6000.0,Астраханская,85.066301,NaN
3,2011.0,Астрахань,12701000,1655.9,38.00,2000.0,4000.0,Астраханская,86.242222,Икрянинский,12620000.0,28.60,55.99,0.0,7930.0,Астраханская,85.066301,2011.0
4,2012.0,Астрахань,12701000,1647.4,38.00,6290.0,18033.0,Астраханская,86.242222,Икрянинский,12620000.0,28.60,15.60,400.0,6960.0,Астраханская,85.066301,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2012.0,Южно-Сахалинск,64701000,140.8,12.75,200.0,1900.0,Сахалинская,77.693897,Корсаковский,64716000.0,9.20,99.58,0.0,1100.0,Сахалинская,82.107766,2011.0
344,2013.0,Южно-Сахалинск,64701000,135.9,13.87,2400.0,8200.0,Сахалинская,77.693897,Корсаковский,64716000.0,27.20,99.58,0.0,700.0,Сахалинская,82.107766,2011.0
345,2014.0,Южно-Сахалинск,64701000,244.3,17.73,2900.0,3000.0,Сахалинская,77.693897,Корсаковский,64716000.0,10.50,6.24,0.0,700.0,Сахалинская,82.107766,NaN
346,2015.0,Южно-Сахалинск,64701000,0.0,19.54,1700.0,5800.0,Сахалинская,77.693897,Корсаковский,64716000.0,0.00,7.30,0.0,36800.0,Сахалинская,82.107766,NaN


In [30]:
def check_years_completeness(oktmo, df_events_full, required_years):
    municipality_data = df_events_full[df_events_full['oktmo'] == oktmo]
    existing_years = set(municipality_data['year'].unique())
    return existing_years.issuperset(required_years)


df_protests = pd.read_csv("протесты2009_2013_2010t3.csv")
df_protests = df_protests.drop_duplicates(subset=['region_name', 'municipality'])
df_events = pd.read_csv("единицы_без_протестов_2010t3.csv")
df_events_full = pd.read_csv("общие_данные_2010t3.csv")

required_years = {2010.0, 2014.0, 2015.0, 2016.0}


df_events = df_events[df_events['year'].isin([2011, 2012, 2013])].reset_index(drop=True)

features = [
    "Жилье (1000 м2)", "Дороги (%)", "Канализация (1 м)",
    "Водопровод (1 метр)"
]

df_protests = df_protests[['region_name', 'municipality', 'oktmo', 'fprotets_year'] + features].reset_index(drop=True)
df_events = df_events[['region_name', 'municipality', 'oktmo'] + features].reset_index(drop=True)


exclude_dict = {
    'Брянск': ['Брянский', 'Сельцо', 'Фокино', 'Титовское'],
    'Вологда': ['Вологодский'],
    'Воронеж': ['Хохольский', 'Семилукский', 'Рамонский', 'Новоусманский', 'Каширский'],
    'Казань': ['Лаишевский', 'Пестречинский', 'Высокогорский', 'Зеленодольский', 'Верхнеуслонский'],
    'Краснодар': ['Динский', 'Красноармейский', 'Тахтамукайский'],
    'Омск': ['Омский'],
    'Тамбов': ['Тамбовский', 'Котовск'],
    'Тула': ['Щекинский', 'Дубенский', 'Алексин', 'Ясногорский', 'Веневский', 'Киреевский', 'Ленинский'],
    'Ярославль': ['Ярославский'],
    'Владивостокский': ['Хасанский', 'Надеждинский', 'Артемовский', 'Шкотовский', 'Большой Камень', 'Фокино'],
    'Южно-Сахалинск': ['Корсаковский', 'Анивский', 'Холмский', 'Долинский'],
    'Магадан': ['Ольский', 'Хасынский'],
    'Саранск': ['Рузаевский', 'Лямбирский', 'Кочкуровский'],
    'Тюмень': ['Тюменский', 'Нижнетавдинский'],
    'Липецк': ['Грязинский', 'Липецкий', 'Добровский'],
    'Астрахань': ['Приволжский', 'Икрянинский', 'Наримановский', 'Харабали', 'Верхний'],
    'Владимир': ['Собинский', 'Судогодский', 'Камешковский', 'Суздальский'],
    'Екатеринбург': ['Первоуральск', 'Дегтярск', 'Полевской', 'Сысертский', 'Белоярский', 'Березовский', 'Верхняя Пышма'],
    'Ижевск': ['Завьяловский', 'Камское'],
    'Киров': ['Оничевский', 'Орловский', 'Юрьянский', 'Слободской', 'Кирово-Чепецкий'],
    'Курган': ['Кетовский', 'Варгашинский', 'Каргапольский'],
    'Нальчик': ['Чегемский', 'Черекский'],
    'Самара': ['Волжский', 'Новокуйбышевск'],
    'Ставрополь': ['Шпаковский'],
    'Уфа': ['Уфимский', 'Кармаскалинский', 'Иглинский', 'Чишминский'],
    'Махачкала': ['Карабудахкентский', 'Каспийск', 'Буйнакский', 'Кумторкалинский', 'Новолакский'],
    'Саратов': ['Саратовский', 'Энгельсский', 'Марксовский', 'Энгельс'],
    'Томск': ['Томский'],
    'Ульяновск': ['Ульяновский', 'Новоульяновск', 'Чердаклинский'],
    'Волгоград': ['Городищенский', 'Дубовский', 'Среднеахтубинский', 'Светлоярский', 'Калачевский'],
    'Магас': ['Назрановский', 'Пригородный'],
    'Мурманск': ['Североморск', 'Кольский'],
    'Новосибирск': ['Новосибирский', 'Кольцово', 'Бердск', 'Обь'],
    'Петрозаводский': ['Прионежский'],
    'Ростовский': ['Борисоглебский', 'Переяславь-Залесский', 'Юрьев-Польский', 'Ильинский', 'Гаврилов-Ямский'],
    'Рязань': ['Рязанский'],
    'Смоленск': ['Смоленский'],
    'Чебоксары': ['Чебоксарский', 'Новочебоксарск', 'Звениговский'],
    'Челябинский': ['Сосновский', 'Копейский', 'Красноармейский'],
    'Грозный': ['Урус-Мартановский', 'Грозненский', 'Аргун'],
    'Белгород': ['Белгородский'],
    'Иваново': ['Ивановский', 'Кохма'],
    'Пенза': ['Заречный', 'Пензенский', 'Бессоновский'],


}


excluded_pairs = set()
for protest, events in exclude_dict.items():
    for event in events:
        excluded_pairs.add((protest, event))

# Проверка на вырожденные признаки
combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
variances = combined_all.var()
threshold = 1e-8
degenerate = variances[variances <= threshold].index.tolist()
if degenerate:
    print(f"Удаляем вырожденные признаки: {degenerate}")
    features = [f for f in features if f not in degenerate]
else:
    print("Вырожденных признаков нет.")

combined_all = pd.concat([df_protests[features], df_events[features]], axis=0)
cov_matrix = np.cov(combined_all.T)
cov_inv = np.linalg.inv(cov_matrix)

# Matching внутри каждого региона с усиленной проверкой ограничений
matches_list = []

for region in df_protests['region_name'].unique():
    sub_protests = df_protests[df_protests['region_name'] == region].reset_index(drop=True)
    sub_events = df_events[df_events['region_name'] == region].reset_index(drop=True)
    
    # Фильтрация событий: полнота данных + проверка на исключения
    valid_events = []
    for _, event_row in sub_events.iterrows():
        event_name = event_row['municipality']
        oktmo = event_row['oktmo']
        
        # Проверка полноты данных
        has_full_data = check_years_completeness(oktmo, df_events_full, required_years)
        
        # Проверка, что нет ни одного протеста, для которого это событие запрещено
        is_allowed = all(
            (protest_name, event_name) not in excluded_pairs
            for protest_name in sub_protests['municipality']
        )
        
        if has_full_data and is_allowed:
            valid_events.append(event_row)
    
    if not valid_events:
        print(f"В регионе {region} нет подходящих муниципалитетов событий")
        continue
    
    sub_events_valid = pd.DataFrame(valid_events).reset_index(drop=True)
    
    n_p = sub_protests.shape[0]
    n_e = sub_events_valid.shape[0]
    
    if n_e == 0:
        continue
    
    dist_mat = np.zeros((n_p, n_e))
    
    # Заполнение матрицы расстояний
    for i in range(n_p):
        x = sub_protests.loc[i, features].values
        for j in range(n_e):
            y = sub_events_valid.loc[j, features].values
            dist_mat[i, j] = mahalanobis(x, y, cov_inv)
    
    # Венгерский алгоритм
    row_ind, col_ind = linear_sum_assignment(dist_mat)
    
    # Сбор результатов
    matched_p = sub_protests.iloc[row_ind].reset_index(drop=True)
    matched_e = sub_events_valid.iloc[col_ind].reset_index(drop=True)
    
    matched = matched_p.join(
        matched_e,
        lsuffix="_protest",
        rsuffix="_event"
    )
    matched["mahalanobis_distance"] = dist_mat[row_ind, col_ind]
    matches_list.append(matched)




df_matches = pd.concat(matches_list, axis=0).reset_index(drop=True)
df_matches.to_csv('прототип_данных_2010t3M2.csv', index=False)
df_matches

Вырожденных признаков нет.
В регионе Чеченская нет подходящих муниципалитетов событий
В регионе Магаданская нет подходящих муниципалитетов событий


,region_name_protest,municipality_protest,oktmo_protest,fprotets_year,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_event,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,mahalanobis_distance
0,Хакасия,Абакан,95701000,2012.0,2.1,0.00,180.0,1946.0,Хакасия,Первомайский,95615440,0.0,0.00,0.0,1866.0,0.254525
1,Архангельская,Архангельск,11701000,2011.0,538.2,2.94,2400.0,5900.0,Архангельская,Котласский,11627000,44.9,5.11,1352.0,3408.0,1.585701
2,Астраханская,Астрахань,12701000,2011.0,1655.9,38.00,2000.0,4000.0,Астраханская,Камызякский,12625000,42.8,2.27,512.0,972.0,2.843375
3,Алтайский,Барнаул,1701000,2011.0,86.0,0.00,100.0,5400.0,Алтайский,Родинский,1636000,4.7,0.00,0.0,5200.0,0.154212
4,Белгородская,Белгород,14701000,2011.0,139.1,0.00,0.0,100.0,Белгородская,Алексеевский,14605000,61.3,0.00,0.0,100.0,0.053495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56,Ханты-Мансийский,Ханты-Мансийск,71871000,2011.0,83.0,0.00,0.0,1195.0,Ханты-Мансийский,Нижневартовский,71819000,39.4,0.00,0.0,1200.0,0.030038
57,Чувашская,Чебоксары,97701000,2011.0,96.3,0.00,2000.0,7800.0,Чувашская,Канаш,97707000,12.0,0.00,800.0,800.0,2.365169
58,Челябинская,Челябинский,75701000,2011.0,131.3,0.00,3400.0,14000.0,Челябинская,Розинское,75633156,0.0,0.00,4550.0,7900.0,2.567516
59,Забайкальский,Чита,76701000,2011.0,0.0,0.00,500.0,6400.0,Забайкальский,Забайкальский,76612000,0.8,7.81,800.0,2790.0,1.261249


In [31]:
df_2012 = pd.read_csv('прототип_данных_2010t3M2.csv')
df_2011 = pd.read_csv('без_протестов2009_2010t3.csv')

df_2012 = df_2012[df_2012["mahalanobis_distance"] <= 10].reset_index(drop=True)

merged_df = pd.merge(
    df_2012,
    df_2011,
    left_on=['municipality_protest', 'oktmo_protest'],
    right_on=['municipality', 'oktmo'],
    how='inner'
)

from2012 = merged_df[[
    'municipality_protest', 'oktmo_protest',
    'Жилье (1000 м2)_protest', 'Дороги (%)_protest',
    'Канализация (1 м)_protest', 'Водопровод (1 метр)_protest',
    'municipality_event', 'oktmo_event',
    'Жилье (1000 м2)_event', 'Дороги (%)_event',
    'Канализация (1 м)_event', 'Водопровод (1 метр)_event', 
    'region_name_protest', 
    'region_name_event', 'fprotets_year'
]]


from2012.to_csv('связка_махаланобис_2010t3М2.csv', index=False)
from2012

,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_protest,region_name_event,fprotets_year
0,Астрахань,12701000,1655.9,38.00,2000.0,4000.00,Камызякский,12625000,42.80,2.27,512.0,972.0,Астраханская,Астраханская,2011.0
1,Белгород,14701000,139.1,0.00,0.0,100.00,Алексеевский,14605000,61.30,0.00,0.0,100.0,Белгородская,Белгородская,2011.0
2,Брянск,15701000,98.2,0.00,1200.0,2500.00,Новозыбковский,15640000,0.00,0.00,660.0,0.0,Брянская,Брянская,2011.0
3,Владивостокский,5701000,99.4,0.00,0.0,2568.50,Михайловский,5620000,1.19,0.00,0.0,2544.0,Приморский,Приморский,2011.0
4,Владимир,17701000,55.6,34.16,0.0,500.00,Округ,17735000,59.80,34.88,0.0,582.0,Владимирская,Владимирская,2012.0
5,Волгоград,18701000,352.1,30.01,545.1,19324.30,Старополтавский,18652000,0.10,0.00,0.0,21900.0,Волгоградская,Волгоградская,2011.0
6,Вологда,19701000,266.0,0.00,67.0,1070.00,Тотемский,19646000,49.00,0.00,50.0,1000.0,Вологодская,Вологодская,2011.0
7,Воронеж,20701000,182.6,0.00,500.0,3500.00,Панинский,20635000,4.40,0.00,400.0,5000.0,Воронежская,Воронежская,2011.0
8,Екатеринбург,65701000,258.0,0.00,500.0,4800.00,Горноуральский,65717000,10.80,0.00,600.0,4960.0,Свердловская,Свердловская,2011.0
9,Ижевск,94701000,44.3,64.61,600.0,2000.00,Глазовский,94610000,5.30,63.97,532.0,1430.0,Удмуртская,Удмуртская,2011.0


In [32]:
df = pd.read_csv('общие_данные_2010t3.csv')

df = df[['year', 'municipality', 'oktmo', 'Жилье (1000 м2)', 'Дороги (%)', 'Канализация (1 м)', 'Водопровод (1 метр)',  'region_name',
        'Процент (рус)',
        ]]
df = df.drop_duplicates(subset=['year', 'municipality', 'oktmo'])


df = df[df['year'].isin([2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
events = df
from2012 = pd.read_csv('связка_махаланобис_2010t3М2.csv')


final = pd.DataFrame()
for _, link_row in from2012.iterrows():
    

    protest_filter = (
        (events['municipality'] == link_row['municipality_protest']) &
        (events['oktmo'] == link_row['oktmo_protest'])
    )
    protest_data = events[protest_filter].add_suffix('_protest')
    

    event_filter = (
        (events['municipality'] == link_row['municipality_event']) &
        (events['oktmo'] == link_row['oktmo_event'])
    )
    event_data = events[event_filter].add_suffix('_event')
    

    merged = protest_data.merge(
        event_data,
        left_on='year_protest',
        right_on='year_event',
        how='left'
    ).rename(columns={'year_protest': 'year'})
    

    merged = merged.drop('year_event', axis=1)
    merged['fprotets_year'] = link_row.get('fprotets_year', pd.NA)
    merged.loc[~pd.to_numeric(merged['year'], errors='coerce').astype('Int64').isin([2011, 2012, 2013]),
               'fprotets_year'] = pd.NA
    final = pd.concat([final, merged], ignore_index=True)


final.to_csv('2010t3М2.csv', index=False)
final


,year,municipality_protest,oktmo_protest,Жилье (1000 м2)_protest,Дороги (%)_protest,Канализация (1 м)_protest,Водопровод (1 метр)_protest,region_name_protest,Процент (рус)_protest,municipality_event,oktmo_event,Жилье (1000 м2)_event,Дороги (%)_event,Канализация (1 м)_event,Водопровод (1 метр)_event,region_name_event,Процент (рус)_event,fprotets_year
0,2008.0,Астрахань,12701000,1762.1,6.39,420.0,6906.0,Астраханская,86.242222,Камызякский,12625000.0,13.60,0.00,1300.0,1600.0,Астраханская,64.06976,NaN
1,2009.0,Астрахань,12701000,1739.0,1.08,8230.0,1200.0,Астраханская,86.242222,Камызякский,12625000.0,9.91,0.00,220.0,16575.0,Астраханская,64.06976,NaN
2,2010.0,Астрахань,12701000,1672.9,91.15,6600.0,8800.0,Астраханская,86.242222,Камызякский,12625000.0,11.50,0.00,500.0,11500.0,Астраханская,64.06976,NaN
3,2011.0,Астрахань,12701000,1655.9,38.00,2000.0,4000.0,Астраханская,86.242222,Камызякский,12625000.0,11.20,37.04,500.0,9940.0,Астраханская,64.06976,2011.0
4,2012.0,Астрахань,12701000,1647.4,38.00,6290.0,18033.0,Астраханская,86.242222,Камызякский,12625000.0,11.20,13.70,8.0,6872.0,Астраханская,64.06976,2011.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,2012.0,Южно-Сахалинск,64701000,140.8,12.75,200.0,1900.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,17.10,99.68,400.0,100.0,Сахалинская,78.76494,2011.0
344,2013.0,Южно-Сахалинск,64701000,135.9,13.87,2400.0,8200.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,22.00,0.00,100.0,4500.0,Сахалинская,78.76494,2011.0
345,2014.0,Южно-Сахалинск,64701000,244.3,17.73,2900.0,3000.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,19.70,0.00,0.0,0.0,Сахалинская,78.76494,NaN
346,2015.0,Южно-Сахалинск,64701000,0.0,19.54,1700.0,5800.0,Сахалинская,77.693897,Северо-Курильский,64743000.0,0.00,0.00,0.0,0.0,Сахалинская,78.76494,NaN
